In [2]:
import pandas as pd
import numpy as np
import csv
import sqlite3
import re
import pickle

from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
filename_csv = '/Users/xzhou/github/project_archives/files_lending_club/loan.csv'
date_to_parse = ['issue_d' , 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line']

df=pd.read_csv(filename_csv, date_parser=date_to_parse)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.shape

(887379, 74)

In [7]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [5]:
filter = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 
          'emp_length', 'home_ownership', 'annual_inc', 'loan_status', 
          'purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line', 
          'inq_last_6mths', 'mths_since_last_delinq', 
          'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 
          'revol_util', 'application_type', 'collections_12_mths_ex_med', 
          'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint', 
          'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 
          'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 
          'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 
          'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 
          'inq_fi', 'total_cu_tl','inq_last_12m']

In [6]:
df_smaller = df.filter(items=filter)

In [7]:
df_smaller.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m'],
      dtype='object')

In [1]:
df_clean.isnull().sum()

NameError: name 'df_clean' is not defined

In [9]:
# Drop the filds have 90%+ null values, 
# as filling in the nulls can skew the model

columns_to_drop = [
    'annual_inc_joint', 'dti_joint',  'open_acc_6m', 'open_il_6m',
    'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
    'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
    'inq_fi', 'total_cu_tl', 'inq_last_12m'
]

df_smaller.drop(columns_to_drop, axis=1, inplace=True)

In [10]:
df_smaller.rename(index=str, 
                  columns={"dti": "payment_to_income_ratio_individual"}, 
                  inplace=True)

In [14]:
df_clean = df_smaller.copy()

In [15]:
df_clean.isnull().sum()

loan_amnt                                  0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
emp_length                             44825
home_ownership                             0
annual_inc                                 4
loan_status                                0
purpose                                    0
payment_to_income_ratio_individual         0
delinq_2yrs                               29
earliest_cr_line                          29
inq_last_6mths                            29
mths_since_last_delinq                454312
mths_since_last_record                750326
open_acc                                  29
pub_rec                                   29
revol_bal                                  0
revol_util                               502
application_type                           0
collections_12_mths_ex_med               145
mths_since

In [17]:
excluded = {'Issued', 'Does not meet the credit policy. Status:Charged Off',
            'Does not meet the credit policy. Status:Fully Paid'}
good = {'Current', 'Fully Paid'}
bad = {'Charged Off', 'Default', 'In Grace Period', 
       'Late (16-30 days)', 'Late (31-120 days)'}

def get_status(status):
    """
    Classify loans to either good or bad loans based on their status.
    """
    try: 
        status=status.strip()
        if status in good:
            return "good"
        elif status in bad:
            return "bad"
        else: 
            return "excluded"
    except: 
        return "excluded"

In [18]:
df_clean['loan_status'] = df_clean['loan_status'].apply(lambda x: get_status(x))

In [19]:
df_clean = df_clean[df_clean['loan_status'] != 'excluded']

In [20]:
df_clean.groupby('loan_status').count()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,purpose,payment_to_income_ratio_individual,...,revol_bal,revol_util,application_type,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
loan_status,,,,,,,,,,,,,,,,,,,,,
bad,66668,66668,66668,66668,66668,62973,66668,66668,66668,66668,...,66668,66607,66668,66662,15124,66668,66668,56417,56417,56417
good,809502,809502,809502,809502,809502,769037,809502,809502,809502,809502,...,809502,809104,809502,809452,204054,809502,809502,752226,752226,752226


In [24]:
# Fill the null value for 'mths_since_last_delinq' with zero as zero is default case for majority of loans

fill_with_zero = {
    'mths_since_last_delinq':0,
    'mths_since_last_record':0,
    'collections_12_mths_ex_med':0,
    'mths_since_last_major_derog':0,
    'tot_coll_amt':0
}
            
df_clean.fillna(fill_with_zero,inplace=True)

In [26]:
# Fill the null value for 'revol util' with means

df_clean['revol_util'].fillna(
    value=df_clean['revol_util'].mean(), inplace=True)

In [27]:
# Fill the null value for 'tot_cur_bal' with means

df_clean['tot_cur_bal'].fillna(
    value=df_clean['tot_cur_bal'].mean(), inplace=True)

In [28]:
# Fill the null value for 'total_rev_hi_lim' with means

df_clean['total_rev_hi_lim'].fillna(
    value=df_clean['total_rev_hi_lim'].mean(), inplace=True)

In [30]:
def get_employee_length(length):
    """
    This is to extract numerical value of emp_length
    For simplicity, approximation was used.
    <1 years will be treated as 1 year
    >10 years will be treated as 10 years
    """
    try:
        length = length.strip()
        year = re.findall(r'(\d+)', length)
        return int(year[0]) .
    except:
        return None

In [31]:
df_clean['emp_length'] = df_clean['emp_length'].apply(lambda x: get_employee_length(x))

In [32]:
df_clean['emp_length'].fillna(
    value=df_clean['emp_length'].mean(), inplace=True)

In [33]:
# All Null values have been appropriately handled.

df_clean.isnull().sum()

loan_amnt                             0
term                                  0
int_rate                              0
installment                           0
grade                                 0
emp_length                            0
home_ownership                        0
annual_inc                            0
loan_status                           0
purpose                               0
payment_to_income_ratio_individual    0
delinq_2yrs                           0
earliest_cr_line                      0
inq_last_6mths                        0
mths_since_last_delinq                0
mths_since_last_record                0
open_acc                              0
pub_rec                               0
revol_bal                             0
revol_util                            0
application_type                      0
collections_12_mths_ex_med            0
mths_since_last_major_derog           0
policy_code                           0
acc_now_delinq                        0


In [34]:
# Break down terms of loan into 2 categories.

df_clean = pd.get_dummies(df_clean, prefix=['term'], columns=['term'])

In [36]:
# Break down grades into categories.

df_clean = pd.get_dummies(df_clean, prefix=['grade'], columns=['grade'])

In [38]:
df_clean = pd.get_dummies(df_clean, prefix=['home_ownership'], columns=['home_ownership'])

In [44]:
df_clean[df_clean['loan_status'] == 'bad'].groupby('purpose').count()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,payment_to_income_ratio_individual,delinq_2yrs,earliest_cr_line,inq_last_6mths,...,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
purpose,,,,,,,,,,,,,,,,,,,,,
car,583,583,583,583,583,583,583,583,583,583,...,583,583,583,583,583,583,583,583,583,583
credit_card,11686,11686,11686,11686,11686,11686,11686,11686,11686,11686,...,11686,11686,11686,11686,11686,11686,11686,11686,11686,11686
debt_consolidation,41316,41316,41316,41316,41316,41316,41316,41316,41316,41316,...,41316,41316,41316,41316,41316,41316,41316,41316,41316,41316
educational,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
home_improvement,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482,...,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482
house,406,406,406,406,406,406,406,406,406,406,...,406,406,406,406,406,406,406,406,406,406
major_purchase,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,...,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271
medical,782,782,782,782,782,782,782,782,782,782,...,782,782,782,782,782,782,782,782,782,782
moving,592,592,592,592,592,592,592,592,592,592,...,592,592,592,592,592,592,592,592,592,592


In [ ]:
df_clean = pd.get_dummies(df_clean, prefix=['purpose'], columns=['purpose'])

In [ ]:
def get_earliest_cr_line(string):
    """
    This is to calculate years of credit history.
    Since the data is from 2016, use 2016 minus the 
    starting year to estimate years of credit history.
    
    """
    string = string.strip()
    year = re.findall(r'(\d+)', string)
    return (2016-int(year[0])) 

In [ ]:
df_clean['earliest_cr_line'] = df_clean['earliest_cr_line'].apply(lambda x: get_earliest_cr_line(x))

In [ ]:
def inverse(value):
    try:
        return 1 / value
    except: 
        return 0

In [ ]:
# Take inverse of months since last delinquishing date as this is is inversed related to default prediction

df_clean['inversed_mths_since_last_delinq'] = df_clean['mths_since_last_delinq'].apply(lambda x: inverse(x))

In [ ]:
# Take inverse of months since last record as this is is inversed related to default to default prediction

df_clean['inversed_mths_since_last_record'] = df_clean['mths_since_last_record'].apply(lambda x: inverse(x))

In [5]:
# Take inverse of months since last record as this is is inversed related to default to default prediction.
new_col_name = 'inversed_mths_since_last_major_derog'
old_col_name = 'mths_since_last_major_derog'
df_clean[new_col_name] = df_clean[old_col_name].apply(lambda x: inverse(x))

NameError: name 'df_clean' is not defined

In [ ]:
df_clean.drop(
    ['mths_since_last_delinq', 'mths_since_last_record',
     'mths_since_last_major_derog'], 
    inplace=True, axis=1)

In [ ]:
# Calculate income-to-debt raio. 
# This can result in infinite values when revol_balance is small.

df_clean['income_to_debt_ratio'] = df_clean['annual_inc'] / df_clean['revol_bal']

In [ ]:
# Replace infinite value with N/As.

df_clean['income_to_debt_ratio'] = df_clean['income_to_debt_ratio'].replace(np.inf, np.nan)

In [ ]:
# Drop infinite values since size of it is very small.

df_clean = df_clean[df_clean['income_to_debt_ratio'] >= 0]

In [ ]:
# Break down applicaiton type to different categories.

df_clean=pd.get_dummies(df_clean, prefix=['application_type'], columns=['application_type'])

In [ ]:
pkl_filename= '/Users/xzhou/github/project_files/lending_club/loan3.pkl'

with open(pkl_filename, 'wb') as pklfile:
    pickle.dump(df_clean, pklfile)